In [ ]:
# !which python
# import sys
# print(sys.executable)

!python -m pip install --upgrade pip
# !python -m pip install pycrypto
# !python -m pip uninstall pycrypto --yes
!python -m easy_install pycryptodome
# !python -m easy_install pycryptodomex
!python -m pip install Pillow
!python -m jupyter nbextension enable --py widgetsnbextension --sys-prefix

print('='*20+'\n\nReady to Go!')

# The Death of DES!

Before AES, there was DES, the gold standard of yesteryear (1980s and 1990s). DES stands for *__D__ata __E__ncryption __S__tandard*, quite presumptive and ironic naming in retrospect, which raises the question, is the word "Advanced" Encryption System (AES) going to become a source of amusement in a not-so-distant future?! 

But joking aside, the death of DES came about primarily due NOT to an inherent weakness in its design (though some *better-than-brute-force* academic attacks were discovered later on) but rather because of its short key length! DES was developed by engi-scientists of IBM in close collaboration with the US's National Security Agency -- NSA (later turned out). NSA's involvement was allegedly a double-edge sword: it helped to strengthen DES against a class of mathematical attack techniques (called *differential cryptanalytic* attacks, later re-discovered), but at the same time, weakened the cipher against brute-force attacks, by (again, allegedly) convincing IBM to choose a shorter key length (with the excuse that the implementation can be done on a single chip)! In what follows, we examine DES a bit, and convince ourselves why **DES is dead**. 

### DES Questions:
- __Q0:__ In terms of the number of bytes and bits, what is the *block-size* of DES? 

 *A0:* 

- __Q1:__ In terms of the number of bits, what is the *key length* and the *key-space size* of DES? 

 <font color=gray>*Hint: the following commands may help. Uncomment by removing the `#` sign or pressing `Ctrl+/` on the commented line, and again, recall that `Shift+Enter` or `Ctrl+Enter` executes a cell in `jupyter notebook`.*</font>

_A1:_ 

In [ ]:
from Crypto.Cipher import DES
# print('DES.block_size: {} bytes (what is it in terms of bits?)'.format(DES.block_size))
# print('DES.key_size: {} bytes (but be careful, not all the bits are "free"...)'.format(DES.key_size))

- __Q2:__ Assume we are using a computing machine consisting of **X** processors, each of which can test **Y million** keys per second, (by "testing a key", we mean executing a DES decryption using that key). Suppose we have a DES ciphertext and we know what plaintext it is coming from (say, it is from a standard header of a pdf file). We just don't know the key. We decide to launch a brute-force attack (exhaustive key search). How long is it likely to take before we find the right DES key? Express the result in terms of seconds, hours, days, and years! Once you are clear with the question, finish the following code that produces the answer.
 
 -  <font color=gray> *Hint 1:* Note that we may get lucky and stumble on the right key even in the first attempt. Or, we may be terribly unlucky and succeed on the very last try! But "on average" how much of the the key-space one would explore before the successful hit?</font>
 - <font color=gray> *Hint 2:* Fortunately, trying different keys can be fully parallelised! In the simple sense that, we can partition the key space and assign each subset to each processor.</font>
 - <font color=gray> *Hint 3:* Computing $a^b$ in python is done by `a**b`.</font>


In [ ]:
%reset -f
import ipywidgets as widgets

# first the function:
def DES_Average_Brute_Force_Time(X, Y):
    """Function that computes the average time it takes to "break" DES cipher through brute-force (exhaustive key search), 
    i.e., find the correct key that decrypts a ciphertext to its corresponding plaintext (assuming a pair is known).
    
    Args (input arguments):
        X: (an integer) the number of available processors.
        Y: (a float) the speed of each processor, in terms of  HOW MANY KEYS PER SECOND each can test.  

    Returns (output):
        Nothing. Just prints the "average" (expected value) of the time for a brute-force attack, 
        in SECONDS, HOURS, DAYS, and YEARS. 
    
    """
    try:
        X = int(X)
        Y = float(Y)
        _DES_KEY_LENGTH = 56 # why is it 56 and not 64?!
        BRUTE_FORCE_SECONDS =  2**(_DES_KEY_LENGTH-1)/(X*Y*1000000) # why the minus 1 in the power?
        BRUTE_FORCE_HOURS = 0 # to be edited by you!
        BRUTE_FORCE_DAYS = 0 # to be edited by you!
        BRUTE_FORCE_YEARS = 0 # to be edited by you!
        
        print('Averge duration of a Brute-Force attack on DES:')
        print(' - in terms of seconds: ~ {:.0f}'.format(BRUTE_FORCE_SECONDS))
        print(' - in terms of hours: ~ {:.2f}'.format(BRUTE_FORCE_HOURS))
        print(' - in terms of days: ~ {:.2f}'.format(BRUTE_FORCE_DAYS))
        print(' - in terms years: ~ {:.3f}'.format(BRUTE_FORCE_YEARS))
        
    except ValueError as e:
        print(e)
        print('The number of available processors and the keys/second rate of each processor should be provided!')
    except Exception as err:
        print(err)
        raise
        
    
# now, using the function:

header_message_DES_brute_force = widgets.Label(value='Estimating the average time for a brute-force attack on DES:')

# input widget to get the number of processors 
input_num_processors = widgets.Text(
    value = '100', # typical size of your cluster
    placeholder = 'Enter the number of processors (an integer)!', # inside text when cleared 
    description = 'Number of available processors: ',
    style = {'description_width': 'initial'},
    layout = widgets.Layout(width='90%'),

)

# input widget to get the speed of each processor in terms of number of keys it can test in each second
input_processing_rate = widgets.Text(
    value = '2000', # a typical CPU nowadays is at least 2 GHz clock speed
    placeholder = 'Enter the peoc. speed of each processor in terms of how many million keys it can test in each second', # inside text when cleared 
    description = 'Proc. Speed of each (Million Keys/Sec)',
    style = {'description_width': 'initial'},    
    layout = widgets.Layout(width='90%'),
    
)

input_num_processors_with_desc = widgets.HBox([widgets.Label('Number of Available Processors: '), input_num_processors])

ui = widgets.VBox([header_message_DES_brute_force, input_num_processors, input_processing_rate])
out = widgets.interactive_output(DES_Average_Brute_Force_Time, {'X': input_num_processors, 'Y': input_processing_rate})
display(ui, out)

 - __Q3:__ Suppose you have intercepted a DES-encrypted pdf detailing the buying budget of a company that you are meeting tomorrow at this time to discuss a deal! You know what the first block of the plaintext is (say it is the predictable standard stuff that appears in the header of any (binary) file of a pdf). So you decide to launch a brute-force attack. But you have to do it before tomorrow's meeting, otherwise the information is irrelevant. You do some estimation and realise your PC is not up to the task. But you remember you live in the age of cloud computing! So you visit this website: __[https://aws.amazon.com/ec2/pricing/on-demand/](https://aws.amazon.com/ec2/pricing/on-demand/)__. After some quick deliberation, you choose to use a grid of `g3.4xlarge` instances. Estimate how much money you have to be ready to pay to *guarantee* you can *crack it* in time?  
  - <font color=gray>*Hint1:* Make the assumption that each vCPU is running at about 3 GHz (a better measure is ECU: each ECU is equivalent to about 1 GHz), and each `g3.4xlarge` instance is about `58` ECU. </font>
  - <font color=gray>*Hint2:* Make the assumption that you need about 100 cycles for each DES encryption or decryption.</font>
  - <font color=gray>*Hint3:* This is a rough estimation, so make any necessary assumptions along the way! </font>
  - <font color=gray>*Hint4:* Remember, if you need a calculator, you have a python right here! You may even decide to write a quick script for it! </font>


In [ ]:
# for your scripting!

You are ready to pay the cloud-based price, before you remember that in 2006, a "Cost-Optimized Parallel COde Breaker" that used dedicated FPGAs called __[COPACOBANA](https://en.wikipedia.org/wiki/Custom_hardware_attack)__ won the DES cracking in a single-day challenge and could be built for less than \$10,000 and you heard there was a successor of it called __[COPACOBANA RIVYERA](http://www.sciengines.com/technology-platform/sciengines-hardware/legacy-products/)__. You do a search and find companies like __[https://crack.sh/](https://crack.sh/)__ that provide guaranteed DES cracking for a fee of about \$300. You learn a lesson that DES is dead and stop pushing the allegory too far! 

## 3DES -- OPTIONAL
In a panic mode (mostly by financial industries, understandably), a makeshift solution to increase the key-length of DES was proposed until a better solution is developed. Tripple-DES was the idea. 

 - __Q0:__ What is the effective key-length of 3DES? (Note: due to a technique called "__[meet-in-the-middle attack](https://en.wikipedia.org/wiki/Meet-in-the-middle_attack)__", it is not three times the effective key length of DES!) 
 - __Q1:__ Compared to DES, how long more does a brute-force attack take (e.g. twice, three times, etc.)?
 - __Q2:__ **Optional** Implement your own 3DES from  Crypto.Cipher.DES. You can write it either as a function or a class, or extend the  Crypto.Cipher.DES class, whichever you prefer.  
 *This exercise is optional because we don't want to spend too much time on a make-shift solution, since as we know, AES replaced it and removed (almost) any excuse to use outdated schemes like DES or 3DES.*   